## DRS document
Defining the functions first and then applying that on a selection of suitable document types (ACs and Orders like).

In [1]:
import PyPDF2
import re
import pandas as pd
import math

def get_bookmarks(outlines, pdf, parent_position_in_bookmark_tree):

    def get_bookmark_content(outline, pdf): # il faut améliorer le code pour pouvoir faire avanter dans la page et pas commencer le get_destination_page_number au debut. reutiliser la derniere page aussi
        if pdf.get_destination_page_number(outline)>=0:
            try:
                repattern_title_number_ac = 'AC 2[13759]'+\
                                            '('+\
                                                '\.[0-9]{1,5}[A-Z]?\.?[\s]{1,3}§'+\
                                                    '|'+\
                                                '(?= MG [0-9]{1,2}\.?)'+\
                                            ')' 
                repattern_title_text_ac = '('+\
                                            '2[13759]\.[0-9]{1,5}\.?'+\
                                            ' (\(A[A-Za-z 0-9-]*\))?' +\
                                                    '|'+\
                                            'MG [0-9]{1,2}\.?'+\
                                            ')'      
                                                                                                                         
                repattern_title_prefix = '(VOLUME|Volume|Section|SECTION|section|PART|SUBPART|CHAPTER|chapter|APPENDIX)+[\s]{1,2}'
                repattern_greek = '[ivx]{2,3}|[IVX]{2,3}'
                repattern_title_prevent_firstletter_mismatch = '(?!([A-Z]{2,10}|[a-z]{2,10}))'    
                repattern_title_number_basic = '[\(]?('+repattern_greek+'|[1-9][0-9]{0,3}|[A-Za-z])'+repattern_title_prevent_firstletter_mismatch
                repattern_title_delimiter = '(-|\)|\.)'

                repattern_title_number ='('+\
                                            repattern_title_number_ac+\
                                        '|'+\
                                            '('+repattern_title_prefix+')?'+\
                                            '('+\
                                                repattern_title_delimiter.join([repattern_title_number_basic, 
                                                                                repattern_title_number_basic, 
                                                                                '[ ]?'+repattern_title_number_basic, 
                                                                                repattern_title_number_basic])+\
                                            '|'+\
                                                repattern_title_delimiter.join([repattern_title_number_basic, 
                                                                                repattern_title_number_basic, 
                                                                                '[ ]?'+repattern_title_number_basic])+\
                                            '|'+\
                                                repattern_title_delimiter.join([repattern_title_number_basic, 
                                                                                repattern_title_number_basic])+\
                                            '|'+\
                                                repattern_title_delimiter.join([repattern_title_number_basic])+\
                                            ')'+\
                                        ')'+\
                                        '[\s]{0,2}'+repattern_title_delimiter+'?'+'[\s]{0,2}'
                
                repattern_before_title = '(?<=( |\n))'
                
                for offset in [0,-1,-2,1,2]:                
                    line_with_title = None
                    nb_keywords = 5
                    while line_with_title is None and nb_keywords>0:
                        outline_title_cleaned = re.sub('[^a-zA-Z ,/\d\s\.:-]', '.', outline.title)

                        title_shortened_variable_spaces = re.sub('[ ]+', ' ', outline_title_cleaned).split(' ')[:nb_keywords]
                        title_shortened_variable_spaces = title_shortened_variable_spaces[:2]+\
                                                        [keyword[:2]+'-?'.join(keyword[2:]) for keyword in title_shortened_variable_spaces[2:]]
                        title_shortened_variable_spaces = '[ ]*'.join(title_shortened_variable_spaces)

                        repattern_title_line = '('+repattern_before_title+repattern_title_number+')?'+\
                                                title_shortened_variable_spaces+\
                                                '.*(\n|\r|$)'
                        try:
                            line_with_title = re.search(repattern_title_line,
                                                        pdf.pages[max(0,pdf.get_destination_page_number(outline)+offset)].extract_text())
                        except:
                            pass                        

                        if line_with_title is not None:
                            break
                        else:
                            nb_keywords = nb_keywords - 1

                    if line_with_title is not None:
                        break
                    
                sentence_around_title = pdf.pages[max(0,
                                                      pdf.get_destination_page_number(outline)+offset)].extract_text()[line_with_title.span()[0]:\
                                                                                                                        max(line_with_title.span()[1],\
                                                                                                                            line_with_title.span()[0]+len(outline.title))]
                title_number = re.search(repattern_title_number, sentence_around_title)

                title_max_length = 200
                title_text_end_delimiter = '(\.|\:|\?|\n|\r|$)?'
                repattern_title_text = '('+repattern_title_text_ac+')?'+\
                                        '[A-Za-z,\/\-\s0-1\(\)\[\]]'+\
                                        '{3,'+title_max_length+'}'+title_text_end_delimiter# [\s]{1,3}
                if title_number is not None:
                    title_text = re.search(repattern_title_text, 
                                           sentence_around_title[len(title_number.group(0)):])
                    string_after_title = title_text.string[title_text.span()[1]:
                                                            (title_text.span()[1]+30)]

                    bookmark_content = [outline.title,
                                        str(pdf.get_destination_page_number(outline)),
                                        str(pdf.get_destination_page_number(outline)+offset),
                                        title_number.group(0),
                                        title_text.group(0),
                                        string_after_title,
                                        str(nb_keywords)
                                        ]
                    pass
                else:
                    title_text = re.search(repattern_title_text, sentence_around_title[len(title_number.group(0)):])
                    string_after_title = title_text.string[title_text.span()[1]:
                                                           (title_text.span()[1]+30)]

                    bookmark_content = [outline.title,
                                        str(pdf.get_destination_page_number(outline)),
                                        str(pdf.get_destination_page_number(outline)+offset),
                                        '',
                                        title_text.group(0),
                                        string_after_title,
                                        str(nb_keywords)
                                        ]
                    pass
            except:
                bookmark_content = [outline.title,
                                    pdf.get_destination_page_number(outline),
                                    '',
                                    '',
                                    '',
                                    '',
                                    ''
                                    ]
                pass
        else:
            bookmark_content = [outline.title,
                                pdf.get_destination_page_number(outline),
                                '',
                                '',
                                '',
                                '',
                                ''
                                ]
            pass

        
        return bookmark_content


    headers = ['position in bookmark tree',
                'original pdf bookmark title (outline.title)',
                'original pdf bookmark page number',
                'corrected pdf bookmark page number',
                'extracted title number',
                'extracted title text',
                'first 20 characters after title',
                'nb_keywords for title search']

    df_content = pd.DataFrame(columns=headers,
                            dtype="string")

    for n, outline in enumerate(outlines):
        if parent_position_in_bookmark_tree=='':
            current_position_in_bookmark_tree = str(n)
        else:
            current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n)

        if type(outline) == PyPDF2.generic._data_structures.Destination:
            df_thisbookmark_content = get_bookmark_content(outline, pdf)
            df_thisbookmark_content = [current_position_in_bookmark_tree]+df_thisbookmark_content

            try:
                df_thisbookmark_content = pd.DataFrame(dict(zip(headers, df_thisbookmark_content)),
                                                    dtype="string", 
                                                    index=[0])
            except UnicodeDecodeError:
                df_thisbookmark_content[1] = df_thisbookmark_content[1].decode('utf-8','ignore')+' (Python Exception: title UnicodeDecodeError)'
                df_thisbookmark_content = pd.DataFrame(dict(zip(headers, df_thisbookmark_content)),
                                                    dtype="string", 
                                                    index=[0])

            df_content = pd.concat([df_content, 
                                    df_thisbookmark_content], 
                                    ignore_index=True)
        elif type(outline) == list:
            extracted_content = get_bookmarks(outline, pdf, current_position_in_bookmark_tree)

            df_content = pd.concat([df_content, 
                                    extracted_content], 
                                    ignore_index=True)
        else:
            raise Exception
        
    return df_content

# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__083AD53732DC0D4686258347005037DF.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__99C827DB9BAAC81B86256B4500596C4E.pdf") # AC 43-13
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\AC\pdf\Current__C7CCE9FCA6D7E34786257D41004C3E63.pdf") # AC 29
# drs_doc = PyPDF2.PdfReader(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\raw data\ORDER_8900.1\pdf\Current__DRSDOCID166392477020230614171412.pdf") # Order

# import glob, os
# matching_file = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'raw data', '**','**','*5934B72DA528D2CC862585230041269E*'))
# drs_doc = PyPDF2.PdfReader(matching_file[0])
# df_content = get_bookmarks(outlines = drs_doc.outline[8:], pdf = drs_doc, parent_position_in_bookmark_tree='') # [8:]

In [2]:
import os
import glob
import pandas as pd
import PyPDF2
import datetime

drs_doc_types = ['AC',
                 'ORDER_8300.10',
                 'ORDER_8400.10',
                 'ORDER_8700.1',
                 'ORDER_8740.1',
                 'ORDER_8900.1',
                 'ORDERS']

df_all_content = None

for drs_doc_type in drs_doc_types:
    df_index = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'index',drs_doc_type+'*.parquet'))[-1]
    df_index = pd.read_parquet(df_index)

    for index, row in df_index.iterrows():

        matching_file = glob.glob(os.path.join(os.getcwd(), 'data', 'DRS', 'raw data', drs_doc_type, 'pdf','*'+row['documentGuid']+'*.pdf'))
        if len(matching_file)>0:
            if row['drs:status']=='Current':
                if 'drs:docID'  in row.index:
                    print(row['drs:docID']+': '+row['documentGuid'])
                elif 'drs:documentNumber' in row.index:
                    print(row['drs:documentNumber']+': '+row['documentGuid'])

                drs_doc = PyPDF2.PdfReader(matching_file[0])
                df_content = get_bookmarks(outlines = drs_doc.outline, pdf = drs_doc, parent_position_in_bookmark_tree='')

                if len(df_content)>0:                    
                    df_content.insert(loc=0, column='documentGuid', value=row['documentGuid'])
                    if df_all_content is not None:
                        df_all_content = pd.concat([df_all_content, 
                                                    df_content], 
                                                    ignore_index=True)
                    else:
                        df_all_content = df_content
                    print('   -> '+str(df_all_content.shape))

                
df_all_content.to_parquet(os.path.join(os.getcwd(), 
                                       'data', 
                                       'DRS', 
                                       'extracted_text_for_'+'-'.join(drs_doc_types)+'-'+datetime.datetime.now().strftime("%Y%m%d - %H%M%S")+'.parquet'))

Unknown destination: AC00-1.1B_FBF.pdf [0, 1]


AC 00-1.1B: 083AD53732DC0D4686258347005037DF
   -> (295, 10)
AC 00-31A: 9575F5C749A8B17F862569E0007769BB
AC 00-33A: CB121E54027E25FF86256F94005ED10D
AC 00-34A: B257578832DD5DD8862569D20070503C
AC 00-41B: 9F97E7B14083DAFF862569BA00688F1C
AC 00-44II: 9BD6AC903BF6335186256A0600713E79
AC 00-46F: FCDFC09CD36D2CC9862586C3005DAE49
AC 00-56B: 62EF19FE4FBB52DE86257E54005160A9
   -> (405, 10)
AC 00-58B: 7EC24112E7900D32862575A70072098C
   -> (642, 10)
AC 00-59: D9BCDDA8B9A573E086256A87006B59B1
AC 00-60B: DFA90C1B002C40DB86257C89006A3709
   -> (804, 10)
AC 00-61: 42BAC09511D405C3862569BA00689482
AC 00-63A: 5B3C1602678F7EAF86257CB7004F1BC5
   -> (959, 10)
AC 00-64: B643BE7DDEA4B3AF8625708C006529FC
   -> (1155, 10)
AC 00-65A: DRSDOCID109035315220230815141636
   -> (1202, 10)
AC 00-66: 00D0C1B00EB5062D86257EA1004ED8AA
AC 00-68: 3980A03E840B16508625804100514B47
   -> (1208, 10)
AC 00-69: ED3B543BF158B729862581C40069678B
   -> (1249, 10)
AC 00-70: 4F020063717FF637862583A9005D6538
   -> (1272, 10)
AC 0

unknown widths : 
[0, IndirectObject(18269, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18273, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18277, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18281, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18269, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18273, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18277, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18281, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18269, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18273, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18277, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18281, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18269, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18273, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18277, 0, 2341345907024)]
unknown widths : 
[0, IndirectObject(18281, 0, 23413459

   -> (17095, 10)
AC 25-12: 7733BBE1CEAF1C42862569D00076A402
AC 25-13: 4B385BB9E4E9C4DA862569D00076ACF7
AC 25-15: 72C371337E518B46862569B200752610
   -> (17099, 10)
AC 25-16: 602B51FA58EA96DF862569D00076AEB0
AC 25-17A: 286D3183A8A1D575862575C80048412B
   -> (17342, 10)
AC 25-18: 98EA949F24483ECC862569D00076B2CB
AC 25-19A: FAB5C2606A93D0248625792F00514DC8
AC 25-20: DA7E5CC219C98016862569D00076B470
AC 25-22: 1FB752AA5831F3ED862569B200753353
   -> (17350, 10)
AC 25-23: AE48F52AD6FDDF28862569B200782551
   -> (17359, 10)
AC 25-24: 236A33D46E04DA5D862569B2007825DE
   -> (17361, 10)
AC 25-25A: 48295E72E95C87EA86257D8900677582


unknown widths : 
[0, IndirectObject(5146, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5141, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5136, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5131, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5126, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5121, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5116, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5146, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5141, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5136, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5131, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5126, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5121, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5116, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5146, 0, 2341352372752)]
unknown widths : 
[0, IndirectObject(5141, 0, 2341352372752)]
unknown 

...

Unknown destination: 8300.13A_CHG_1 Feedback Form.pdf [0, 1]
Unknown destination: 8300.13A_CHG_1 Feedback Form.pdf [0, 1]


   -> (157872, 10)
8300.16A CHG 1: DRSDOCID170950420220230118153741
   -> (158362, 10)
8400.9: CAB311E5A20467E786256F87007115BB
   -> (158372, 10)
8400.13F CHG 1: DRSDOCID196813505720220825140439


Unknown destination: Directives Feedback Form PDF (7-31-19).pdf [0, 1]


   -> (158766, 10)
8400.34 CHG 1: DRSDOCID162119420620221013154136
   -> (158980, 10)
8430.21A: 7CBD73BA7F7416B086256E43006E933B
   -> (158988, 10)
8430.24A: DRSDOCID119334869320230118154355
8520.2F: F190C7374BF43FAF86257D77006C5022
8620.2B: DRSDOCID195441924920230118155200
   -> (159022, 10)
8900.1A: DRSDOCID107950986320221103145846
   -> (159116, 10)
8900.2C: DRSDOCID176796247120230118155742
   -> (165367, 10)
8900.3A: DRSDOCID171001912820230118162646
   -> (165401, 10)
8900.4: DRSDOCID146689453920230118163034
   -> (165428, 10)
9120.1E: DRSDOCID137218892020230803185014
   -> (165454, 10)
9500.25: F38C3AB3D0A0310386257D77006C5017
9550.8: A2772BAA15AB56968625753E0072F164
AM 1100.3G: 9D1111F8D76D866186257D77006C501D
AM 8065.1A: 2ED010C700CDC7EE86257D77006C5019
AM 9000.2B: A0D352AB4E4D300586257D77006C501A
AM 9950.3C: 1D1F63B7D7DDDBD386257D77006C5020
AM 9950.3C: 5D525E739241D98486257D77006C501C
FS 3450.1C: DRSDOCID105559339720230626140923
   -> (165608, 10)
FS 8000.96: DRSDOCID1240351616

c:\Users\victor\anaconda3\envs\faa-nlp-drs\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


#### Debugging

In [1]:
import pandas as pd

# df_all_content = pd.read_parquet(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\extracted_text_for_AC-ORDER_8300.10-ORDER_8400.10-ORDER_8700.1-ORDER_8740.1-ORDER_8900.1-ORDERS.parquet")
df_all_content = pd.read_parquet(r"C:\Users\victor\Documents\DeepLearning\FAA NLP Project\data\DRS\extracted_text_for_AC-ORDER_8300.10-ORDER_8400.10-ORDER_8700.1-ORDER_8740.1-ORDER_8900.1-ORDERS-20231214 - 020735.parquet")

## Non DRS document: AIM
Defining the functions first and then applying that on a selection of Non DRS documents.

In [ ]:
import PyPDF2
import re
import pandas as pd
import math

def get_bookmarks(outlines, pdf, parent_position_in_bookmark_tree):
    headers = ['position in bookmark tree',
                'original pdf bookmark title (outline.title)',
                'original pdf bookmark page number',
                'text'
                ]
    df_content = pd.DataFrame(columns=headers,
                            dtype="string")

    for n, outline in enumerate(outlines[0:-1]):
        if parent_position_in_bookmark_tree=='':
            current_position_in_bookmark_tree = str(n)
        else:
            current_position_in_bookmark_tree = parent_position_in_bookmark_tree+'>'+str(n)

        if type(outline) == PyPDF2.generic._data_structures.Destination:            
            text = drs_doc.pages[drs_doc.get_destination_page_number(drs_doc.outline[n]):
                                drs_doc.get_destination_page_number(drs_doc.outline[n+1])].extract_text()
            
            bookmark_content = [outline.title,
                                str(pdf.get_destination_page_number(outline)),
                                str(pdf.get_destination_page_number(outline)+offset),
                                text
                                ]
            
            df_thisbookmark_content = pd.DataFrame(dict(zip(headers, bookmark_content)),
                                                dtype="string", 
                                                index=[0])
                
        elif type(outline) == list:
            extracted_content = get_bookmarks(outline, pdf, current_position_in_bookmark_tree)

            df_content = pd.concat([df_content, 
                                    extracted_content], 
                                    ignore_index=True)
        
    return df_content

In [1]:
import os
import glob
import pandas as pd
import PyPDF2
import datetime

raw_documents = glob.glob(os.path.join(os.getcwd(), 'data', 'NotInDRS', 'raw data', '*.pdf'))

for raw_document in raw_documents:
    drs_doc = PyPDF2.PdfReader(raw_document)
    df_content = get_bookmarks(outlines = drs_doc.outline, pdf = drs_doc, parent_position_in_bookmark_tree='')

    if len(df_content)>0:                    
        df_content.insert(loc=0, column='documentGuid', value=row['documentGuid'])
        if df_all_content is not None:
            df_all_content = pd.concat([df_all_content, 
                                        df_content], 
                                        ignore_index=True)
        else:
            df_all_content = df_content
        print('   -> '+str(df_all_content.shape))

                
df_all_content.to_parquet(os.path.join(os.getcwd(), 
                                       'data', 
                                       'NotInDRS', 
                                       'extracted_text_for_DocNotInDRS'+'-'+datetime.datetime.now().strftime("%Y%m%d - %H%M%S")+'.parquet'))

KeyboardInterrupt: 

## Patterns to extract FAA paragraphs

* For Useful FAA documents:
  * AC: OK

  * ADFRAWD: 
    * REGEX:
    * EXAMPLE:

  * ADFREAD:
    * REGEX:
    * EXAMPLE:

  * ADNPRM:
    * REGEX:
    * EXAMPLE:

  * AFS_FFS_UPDATEPUB:
    * REGEX:
    * EXAMPLE:

  * AFS_FOCUS_TEAMS:
    * REGEX:
    * EXAMPLE:

  * AIRCRAFT_MASTER_SCHEDULE:
    * REGEX:
    * EXAMPLE:

  * ALERTS:
    * REGEX:
    * EXAMPLE:

ULTS WERE, NO ABNORMAL WEAR DETECTED. THIS ENGINE TT O/H WAS 57 HOURS. NOTE; THERE WAS A
SIMILAR FAILURE OF AN OTHER ENGINE OF THIS TYPE JUST 3 MONTHS PRIOR TO THIS ONE, THE TTSOH 123 HRS.
WILL UPDATE THIS REPORT WITH A PROBABLE CAUSE AND RECOMMENDATION WHEN MORE INFORMATION
AVAILABLE. (K)
2007FA0000295 CESSNA
CONT
BEARING
WORN
3/20/2007
414A
TSIO520*
TURBOCHARGER
FAILURE OF RT TURBOCHARGER SEAL AND CENTER BEARING CAUSED SMOKE OUT OF EXHAUST AND
SQUEALING NOISE RESULTING IN THE FLIGHT CREW TO FEATHER PROPELLER AND SHUTDOWN THE ENGINE.
CAUSE OF FAILURE DO TO WEAR. (K)
FMOR477K
CESSNA
CONT
UPLOCK HOOK
DISTORTED
4/9/2007
414A
TSIO520*
57412025
MLG
PILOT R
-------------

RCRAFT LT THE FACTORY.
2007FA0000341 CIRRUS
CONT
RELAY
FAILED
4/16/2007
SR22
IO550N
T/E FLAP
DURING A TRAINING FLIGHT, THE FLAPS BECAME STUCK IN THE DOWN POSITION. THE AIRCRAFT RETURNED TO
THE AIRPORT WITHOUT INCIDENT AND THE AIRCRAFT WAS SENT TO MAINTENANCE FOR TROUBLESHOOTING.
MAINTENANCE DISCOVERED THE FLAPS DOWN RELAY FAILED.
2007FA0000337 CIRRUS
CONT
CONTROL SYSTEM
OUT OF RIG
4/18/2007
SR22
IO550N
AILERON SYS
DURI
    
----------

WAS EVIDENT, WITH LITTLE OR NO PATTERN OF HONEYCOMB IN THE ADHESIVE IN/ ON THE INNER
SURFACE MATERIAL(S). NO EVIDENCE OF POST PRODUCTION REPAIRS WAS APPARENT. ATL MIDO WILL
PROCESS FOR POSSIBLE NON-CONFORMANCE. (K)
2007FA0000229 ISRAEL
GARRTT
VALVE
CRACKED
3/8/2007
1124A
TFE731*
653014
RT LWR FUEL CELL
CRACK IN INTERCONNECT VALVE MOTOR HOUSING. FOUND DURING BAG REMOVAL AND REINSTALLATION.
CAUSE OF CRACK, UNKNOWN. (K)
2007FA0000346 LEAR
GARRTT
ADC
MALFUNCTIONED
1/22/2007
35A
TFE731*
702490031304
DURING P

  * AT_JTA:
    * REGEX:
    * EXAMPLE:

  * BULLETINS:
    * REGEX:
    * EXAMPLE:


  * CFRFRSFAR:
    * REGEX:
    * EXAMPLE:


  * CFRFRSFAR:
    * REGEX:
    * EXAMPLE:

  * ELOS:
    * REGEX:
    * EXAMPLE:

  * FAR:
    * REGEX:
    * EXAMPLE:

SECTION:   Sec. 60.25
Amendment Number: Initial, Effective Date: 10/30/2006

TITLE:   Operation with missing, malfunctioning, or inoperative components.

SECTION RULE:   (a) No person may knowingly use or allow the use of or misrepresent the capability of an FSTD for any maneuver, procedure, or task that is to be accomplished to meet training, evaluation, or flight experience requirements of this chapter for flight crewmember certification or qualification when there is a missing, malfunctioning, or inoperative (MMI) component that is required to be present and correctly operate for the satisfactory completion of that maneuver, procedure, or task.
(b) Each MMI component as described in paragraph (a) of this section, or any MMI component installed and required to operate correctly to meet the current Statement of Qualification, must be repaired or replaced within 30 calendar days, unless otherwise required or authorized by the NSPM.
(c) A list of the current MMI components must be readily available in or adjacent to the FSTD for review by users of the device. Electronic access to this list via an appropriate terminal or display in or adjacent to the FSTD is satisfactory. The discrepancy log may be used to satisfy this requirement provided each currently MMI component is listed in the discrepancy log.

NPRM ACTIONS:   Not Applicable.

FINAL RULE ACTIONS:   Not Applicable.
    
  * FSB_REPORTS:
    * REGEX:
    * EXAMPLE:


  * GA_JTA:
    * REGEX:
    * EXAMPLE:


  * INFO:
    * REGEX:
    * EXAMPLE:



An InFO contains valuable information for operators that should help them meet certain administrative, regulatory,
or operational requirements with relatively low urgency or impact on safety.
Subject: Air Carrier Safety and Pilot Training (ACSPT) Aviation Rulemaking Committee
(ARC) Charter and Report
Purpose: This InFO describes where the ACSPT ARC charter and report can be found and
where requested responses to the report should be directed.
Discussion: The FAA convened the ACSPT ARC to meet the requirements set forth in Section
204 of Public Law 111-216. The ACSPT ARC has completed its first report and delivered it to
Congress on July 29th, 2011. The ACSPT ARC charter and report may be found at the following
link: http://www.faa.gov/about/committees/rulemaking/.
The ACSPT ARC is required to produce a second report to be delivered to Congress by July 31st,
2012. This report will include prevalence of use of best practices identified in the first report and
provide recommendations for legislative or regulatory action. The ACSPT ARC is requesting air
carrier industry responses to the report delivered to Congress on July 29, 2011, in order to
support the preparation of this second report and asks that all responses are directed to the
following e-mail address no later than January 31, 2012: acspt.arc@gmail.com.
Contact: Questions or comments regarding this InFO should be directed to the Air
Transportation Division, AFS-200, telephone (202) 267-8166. Questions or comments regarding
the AC


-------------

BJECT: Maneuvers Away From Planned Track in Oceanic Airspace.
Purpose: To inform aircraft operators of the need for pilots to coordinate with the appropriate
air traffic facility when they plan a maneuver away from a cleared route.
Background: Improved aircraft tracking and reporting equipment is coming into use both
onboard trans-oceanic aircraft and in the air traffic facilities controlling trans-oceanic routes.
These new technology applications mean that maneuvers from planned track in excess of
standard Strategic Lateral Offset Procedures automatically are reported to air traffic facilities,
sometimes before flight crews are able to contact the facility regarding their maneuvering. At
present, this can be a problem from a control and airspace coordination standpoint. However, as
capacity enhancements, such as reduced aircraft separation, come into use, maneuvers away
from track without coordination become a greater concern.
Discussion: The trans-oceanic flying environment usually involves long range flying that
transits multiple weather systems. It is normal that flight crews would need to maneuver in
oceanic airspace for the sake of flight safety, but they are required to coordinate such maneuvers
with the appropriate air traffic facility. Oceanic traffic density is increasing, taking advantage of
such technology as enhanced navigation, communication, and in particular surveillance. In this
environment, coordination with air traffic facilities is increasingly important.
Recommended action: Directors of safety, directors of operations (parts 121 and 135), training
managers, and pilots should maintain or refresh their awareness of reporting requirements
involving maneuv
    


  * LEGAL_INTERPRETATIONS:
    * REGEX:
    * EXAMPLE:



    


  * MMEL:
    * REGEX:
    * EXAMPLE:



    


  * MMEL_POLICY_LETTERS:
    * REGEX:
    * EXAMPLE:



    


  * NORSEE:
    * REGEX:
    * EXAMPLE:



    


  * NOTICES:
    * REGEX:
    * EXAMPLE:

super bien

  * NPRM:
    * REGEX:
    * EXAMPLE:

  * OPSS_GUIDANCE:
    * REGEX:
    * EXAMPLE:

  * ORDER_8300.10: OK
    * REGEX:
    * EXAMPLE:

  * ORDER_8400.10: OK
    * REGEX:
    * EXAMPLE:

  * ORDER_8700.1: OK
    * REGEX:
    * EXAMPLE:

  * ORDER_8740.1: OK
    * REGEX:
    * EXAMPLE:

  * ORDER_8900.1: OK
    * REGEX:
    * EXAMPLE:

  * ORDERS: OK
    * REGEX:
    * EXAMPLE:

  * OSR:
    * REGEX:
    * EXAMPLE:

  * OSWG:
    * REGEX:
    * EXAMPLE:

  * OTHER_EFB_CHECKLISTS:
    * REGEX:
    * EXAMPLE:

  * OTHER_EFB_RESEARCH_REPORTS:
    * REGEX:
    * EXAMPLE:

  * OTHER_FAA_90_DAY_SAFETY_REVIEW:
    * REGEX:
    * EXAMPLE:

  * OTHER_INTERNATIONAL_PUBLICATIONS:
    * REGEX:
    * EXAMPLE:

  * OTHER_JOB_AIDS:
    * REGEX:
    * EXAMPLE:

  * OTHER_PS_FEDERAL_AVIATION_ACTS:
    * REGEX:
    * EXAMPLE:

  * OTHER_PS_HANDBOOKS:
    * REGEX:
    * EXAMPLE:

  * OTHER_RCCB:
    * REGEX:
    * EXAMPLE:
    
  * PILOT_QUALIFICATION_CURRICULUM:
    * REGEX:
    * EXAMPLE:

  * POLICY:
    * REGEX:
    * EXAMPLE:


Summary
This policy statement addresses the certification of Title 14 Code of Federal Regulations (CFR)
Part 23 aircraft that include a Wireless Local Area Network (WLAN), using Institute of
Electrical and Electronics Engineers (IEEE) 802.11 a/b/g/n protocols. More specifically, the
policy is to clarify the guidance regarding cabin WLAN equipment installation that creates a
Radio Frequency (RF) network and provides connectivity (i.e., internet connection and email
services) through access point(s) to users with IEEE 802.11 compliant Portable Electronic
Devices (PEDs). The primary concern of the subject WLAN certification is the aircraft systems
immunity and compatibility with RF energy from transmitting PEDs (T-PEDs).
Current Regulatory and Advisory Material
The regulations applicable to the installation of WLAN using IEEE 802.11 protocols are 14 Title
CFR §§ 23.1301, 23.1309(a), (b)(1), and 23.1431(b).
The following Advisory Circulars (ACs) have guidance that is relevant to WLAN installation:
 AC 20-168, Certification Guidance for Installation of Non-Essential, Non-Required
Aircraft Cabin Systems & Equipment (CS&E)
 AC 20-164, Designing and Demonstrating Aircraft Tolerance to Portable Electronic
Devices
 AC 23-1309-1E, System Safety Analysis and Assessment for Part 23 Airplanes
2
Background and Relevant Past Practice
1.
Past FAA certification projects involving WLAN systems focused on the downlink of
information from satellites with limited uplink and transmission from the airplane cabin.
The configuration of transmitters and receivers within the cabin was rigorously controlled
and qualified to airplane installation standards.
2.
The current WLAN installation involves the use of several transmitters from various
consumer electronic equipment (e.g., laptop/tablet computers, smart phones, etc.) whose
transmission specifications and failure modes cannot be rigorously analyzed and tested due
to the unregulated use of these devices on an airplane.
3.
New computer interfaces, networks and data protocols have been developed to use wireless
radio-frequency (RF) links to transfer data. This allows communications without hard-
wired connections. One approach uses IEEE standard 802.11a/b/g/n, which collectively is
known as Wi-Fi, for wireless communication. IEEE 802.11b, g, and n use wideband
digital modulation techniques in the 2400 to 2483.5 MHz band. IEEE 802.11a and n uses
wideband digital modulation techniques in the 5250 to 5350 MHz and 5470 to 5825 MHz
frequency bands. Wireless transmitters using IEEE 802.11a, b. g, or n protocol do not
require FCC licenses and are allowed to have 1 watt transmitter power with up to 6 dBi
antenna gain, which results in 4 watts Effective Isotropic Radiated Power (EIRP) in the
US.
4.
Safety issues related to the installation and use of the wireless RF system within the
airplane include:
 Potential interference with avionics systems whose failure conditions are
classified as major or worse, or any required systems
 Operation of PED and a wireless RF system which is not fully built to airborne
equipment standards
 Vulnerability of airplane systems to intentional or spurious emission of RF energy
5.
An issue paper has been used recently to address the certification and compliance concerns
related to WLAN installation and RF field strength resulting from PED usage.
6.
AC 20-164 was published with guidance on demonstrating that aircraft and installed
systems are tolerant to potential interference from portable electronic devices.
Policy
The policy points out the items to be considered, and methods acceptable to the FAA regarding
the aircraft toleranc
    


  * SAFO:
    * REGEX:
    * EXAMPLE:



 ble to air carriers in meeting their statutory duty to provide service with the highest possible degree of safety in the public
interest. Besides the specific action recommended in a SAFO, an alternative action may be as effective in addressing the safety
issue named in the SAFO.
Subject: Title 14 of the Code of Federal Regulations (14 CFR) Part 121 Operators Flap Misconfiguration
Events
Purpose: This SAFO serves to raise awareness of aircraft misconfigurations on takeoff with an emphasis
on flap position.
Background: The Aviation Safety Information Analysis and Sharing (ASIAS) program recently
concluded an analysis of flap misconfiguration events. This analysis included takeoff attempts with flaps
set at zero and flap movement during the takeoff roll.
Discussion: The ASIAS analysis indicates that while flap misconfiguration events on takeoff are rare,
they do exist. Approximately half of the events for misconfiguration during the takeoff roll resulted in
rejected takeoffs. The other takeoffs were continued with the flaps being moved during the takeoff roll.
The configuration of the airplane by the flightcrew should be in accordance with the operational
checklists. It is imperative that flightcrews exercise discipline in the use and the execution of operational
checklists to prevent aircraft misconfigurations on takeoff.
Recommended Action: Directors of Operations, Directors of Safety, Program Managers, Directors of
Training, Training Center Program Managers, Check Pilots, Training Pilots, Chief Pilots and flightcrews
should be familiar with the information contained in this SAFO. They should work together to ensure that
the content of this SAFO is emphasized in their training programs to raise awareness among their
flightcrews of flap misconfiguration events. This is especially important if procedures are being changed
in light of the new Federal Aviation Administration (FAA) de-icing procedures. Operators should review
their own Aviation Safety Action Program (ASAP) and Flight Operational Quality Assurance (FOQA)
data for flap misconfiguration events. Operators are also reminded to conduct an analysis on any change
as part of their Safety Management System (SMS) processes.
Contact: Questions   


  * SAIB:
    * REGEX:
    * EXAMPLE:



    


  * SAS_AXH_DCT:
    * REGEX:
    * EXAMPLE:



    


  * SAS_DCT:
    * REGEX:
    * EXAMPLE:



    


  * SCFINAL:
    * REGEX:
    * EXAMPLE:


  * SCPROPOSED:
    * REGEX:
    * EXAMPLE:


  * SFAR:
    * REGEX:
    * EXAMPLE:


  * TSO:
    * REGEX:
    * EXAMPLE:


Subject: Personnel Parachute Assemblies and Components
1.
PURPOSE. This technical standard order (TSO) is for manufacturers applying for a TSO
authorization (TSOA) or letter of design approval (LODA). In it, we the Federal Aviation
Administration (FAA), tell you what minimum performance standards (MPS) your personnel
parachute assembly and components must first meet for approval and identification with the
applicable TSO marking.
2.
APPLICABILITY. This TSO affects new applications submitted after its effective date.
a. All prior revisions to this TSO are no longer effective. Generally, we will not accept
applications for the previous revision after the effective date of this TSO. We may do so,
however, up to six months after it, if we know that you were working against the prior MPS
before the new change became effective.
b. Personnel parachute assemblies and components approved under a previous TSOA may
still be manufactured under the provisions of its original approval.
3. REQUIREMENTS. New models of personnel parachute assemblies and components
identified and manufactured on or after the effective date of this TSO must meet the MPS
qualification and documentation requirements in Parachute Industry Association (PIA)
Technical Standard 135 TS-135 Revision 1.4 issued April 22, 2010 “Performance Standards for
Personnel Parachute Assemblies and Components” as modified by appendix 1 of this TSO.
a. Functionality. This TSO’s standards apply to equipment intended to be used as a
reserve or emergency parachute.
b. Failure Condition Classifications.
(1) Lose of t
    





g. Deviations. We have provisions for using alternate or equivalent means of compliance
with the criteria in the MPS of this TSO. If you invoke these provisions, you must show that
your equipment maintains an equivalent level of safety. Apply for a deviation pursuant to Title
14 of the Code of Federal Regulations (CFR) 21.618.
4.
MARKING.
a. Mark at least one major component permanently and legibly with all of the information
in 14 CFR 45.15(b). Mark each article according to Equipment Class(es) from Table 1 above.
b. If the article includes software and/or airborne electronic hardware, then the article part
numbering scheme must identify the software and airborne electronic hardware configuration.
The part numbering scheme can use separate, unique part numbers for software, hardware, and
airborne electronic hardware.
c. You may use electronic part marking to identify software or airborne electronic hardware
components by embedding the identification within the hardware component itself (using
software) rather than marking it on the equipment nameplate. If electronic marking is used, it
must be readily access

  * UNAPPROVED_PARTS_NOTIFICATIONS:
    * REGEX:
    * EXAMPLE:

  * (6) Training on Records.
  * 21. PASSENGER AND CARGO LOADING PROCEDURES. 
  * (1) Loading, based on aircraft configuration, i.e.,....
  * 1. Purpose.
  * Purpose: This InFO informs air carriers of the availability of the Department of Transportation’s (DOT) National Aviation Resource Manual for Quarantinable Diseases. 
  * SUBJECT: DOT